# Data preprocessing

In [1]:
# Import libraries
import pandas as pd
import os
import billboard
import lyricsgenius
import json
import re

In [2]:
# Adding year columns and wrapping up billboard songs (1950-2015)
cwd = os.chdir("D:\Analiza_i_przetwarzanie_danych\Seminarium_magisterskie\data")
all_csv_files = os.listdir()
songs_50_15 = []
for filename in all_csv_files:
    songs = pd.read_csv(filename, header = 0)
    year = os.path.basename(filename)[:-4]
    songs["Year"] = year
    songs_50_15.append(songs)

In [3]:
# Downloading data from years 2016-2020 from billboard library
playlist = 'hot-100-songs'
dates = [2016, 2017, 2018, 2019, 2020]
songs_billboard = []
for y in dates:
    songs = billboard.ChartData(playlist, date=None, year=y, fetch=True, timeout=25)
    songs_billboard.append(songs)

In [4]:
# Preprocessing data from years 2016-2020
song_titles = []
artists = []
for year in songs_billboard:
    for playlist in year:
        song = str(playlist).split(" by ")
        song_titles.append(song[0].strip("\'"))
        artists.append(song[1])
        
# Adding missed song on 87th position in 2016 chart
song_titles.insert(86, 'All the Way Up')
artists.insert(86, 'Fat Joe, Remy Ma and Jay-Z featuring French Montana and Infared')

# Splitting list into same-length elements lists
def list_split(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

split_artists = list(list_split(artists, 100))
split_song_titles = list(list_split(song_titles, 100))

# Creating list of dictionaries for each year to convert them into pandas DataFrame
def dicts_of_dates(lst_of_dates):
    songs = []
    for i in range(len(lst_of_dates)):
        ranger = list(range(1, 101))
        year = lst_of_dates[i]
        dict_ = {'Position': ranger, 'Artist': split_artists[i], 'Song Title': split_song_titles[i], 'Year': [dates[i] for year in range(1, 101)]}
        as_pandas_df = pd.DataFrame(dict_)
        songs.append(as_pandas_df)
    return songs

songs_16_20 = dicts_of_dates(dates)

all_songs = songs_50_15 + songs_16_20
all_time_billboard_wrap_up = pd.concat(all_songs)
all_time_billboard_wrap_up.drop('jPosition', inplace = True, axis = 1) # drop 'jPosition' column
all_time_billboard_wrap_up.reset_index(drop=True, inplace=True) # reset indexing
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950
1,2,Nat King Cole,Mona Lisa,1950
2,3,Anton Karas,Third Man Theme,1950
3,4,Gary and Bing Crosby,Sam's Song,1950
4,5,Gary and Bing Crosby,Simple Melody,1950
...,...,...,...,...
6675,96,Morgan Wallen,More Than My Hometown,2020
6676,97,Luke Combs,Lovin' On You,2020
6677,98,Moneybagg Yo,Said Sum,2020
6678,99,H.E.R. Featuring YG,Slide,2020


# Spotify tokens

# Genius lyrics

In [5]:
# Spliting authors by 'and' and 'Featuring'
artists = list(all_time_billboard_wrap_up.loc[:,"Artist"])
split_artists = []
for artist in artists:
    if any(re.findall(r'and|Featuring', str(artist))):
        result = re.split(r'and|Featuring', str(artist))[0].strip()
        split_artists.append(result)
    else:
        split_artists.append(artist)

all_time_billboard_wrap_up["Split names"] = split_artists
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year,Split names
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,Gordon Jenkins
1,2,Nat King Cole,Mona Lisa,1950,Nat King Cole
2,3,Anton Karas,Third Man Theme,1950,Anton Karas
3,4,Gary and Bing Crosby,Sam's Song,1950,Gary
4,5,Gary and Bing Crosby,Simple Melody,1950,Gary
...,...,...,...,...,...
6675,96,Morgan Wallen,More Than My Hometown,2020,Morgan Wallen
6676,97,Luke Combs,Lovin' On You,2020,Luke Combs
6677,98,Moneybagg Yo,Said Sum,2020,Moneybagg Yo
6678,99,H.E.R. Featuring YG,Slide,2020,H.E.R.


In [6]:
# Lyrics to jsons
genius_token = os.environ.get('genius_token')
genius = lyricsgenius.Genius(genius_token)
songs = list(all_time_billboard_wrap_up.loc[:, "Song Title"])
artists = list(all_time_billboard_wrap_up.loc[:,"Split names"])
song = genius.search_song('Missing Piece', artist = 'Vance Joy')
print(song.lyrics)


Searching for "Missing Piece" by Vance Joy...
Done.
[Verse 1]
I've been waiting for the tides to change
For the waves to send you my way
I see you darling but you pixelate
It gets hard to take these days

[Pre-Chorus]
But we’ll hold the line, I won't let go
'Cause I’ll be there when you can finally make it home
And I don't mind
'Cause we both know
That we'll be fine when you can finally make it home

[Chorus]
Because when I'm in a room with you
That missing piece is found
You know when you're by my side, darling
Nothing can bring us down

[Verse 2]
I remembеr happy wasted days
Summers golden hazе in our eyes
Lifting you above the breaking waves
Memories floating back to my mind

[Pre-Chorus]
You said hold the line, I won't let go
Cause I'll be there when you can finally make it home
And I don’t mind
’Cause we both know
That we'll be fine when you can finally make it home
[Chorus]
Because when I’m in a room with you
That missing piece is found
You know when you're by my side, darling
No

'\nfor i in range(0, 10):\n    artist = genius.search_artist(artists[i], max_songs=0)\n    song = genius.search_song(songs[i], artist.name)\n    song = artist.song(songs[i])\n    #artist.save_lyrics()\n'